### 머신러닝 문제
1. bike.csv 파일을 불러오세요.

* datetime : 기록 일시(1시간 간격)
* season : 계절(봄 : 1, 여름 : 2, 가을 : 3, 겨울 : 4)
* holiday : 공휴일 여부(0 : 공휴일 아님, 1 : 공휴일)
* workingday : 근무일 여부(0 : 근무일 아님, 1 : 근무일)
* weather : 날씨(1 : 맑음, 2 : 옅은 안개, 3 : 약한 눈 or 약한 번개 or 흐림, 4 : 폭우 or 짙은 안개)
* temp : 실제 온도
* atemp : 체감 온도
* humidity : 상대 습도
* windspeed : 풍속
* casual : 등록되지 않은 사용자(비회원 수)
* registered : 등록된 사용자(회원) 수
* count : 자전거 대여 수량

In [14]:
import pandas as pd
bike=pd.read_csv('bike.csv')

2. 자전거 대여 수요 예측을 해주는 프로그램을 만들어보세요. 다양한 모델을 활용해서 가장 높은 정확도를 이끌어보세요.  
(count열이 자전거의 수요값입니다. 다양한 특성을 활용해서 자전거의 수요를 예측해보세요.)

수치값에 대한 예측이니 분류모델이 아닌 회귀모델

In [4]:
bike.info()
# nan값은 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [5]:
for i in list(bike)[1:11]:
    print(bike[i].max())
# 수치의 차이가 크지 않아서 스케일링 안해도 될거같음
# datetime이랑 target인 count 제외하고 슬라이싱

4
1
1
4
41.0
45.455
100
56.9969
367
886


랜덤포레스트 회귀 모델

In [6]:
data=[]
target=[]

for i in range(len(bike)):
    a=list(bike.iloc[i,1:11]) # [i,:11]target을 제외한 i행의 값을 리스트로 묶음
    # [i,1:11] 타임스탬프는 미분이 안되서 그런건지 학습이 안됨.
    b=bike.iloc[i,11] # i행의 count에 접근
    # i행의 count값은 i행의 다른 값들에 의해 결정되므로 data와 target의 값을 위처럼 만듬
    data.append(a) # 리스트로 묶인 a를 data라는 빈리스트에 넣으면서 2차원 리스트가 됨
    target.append(b) # 1차원 리스트가 됨. target이 count하나만 요구(1차원 값을 요구)

import numpy as np
data=np.array(data) # 행렬로 변환
target=np.array(target)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data,target,test_size=0.1)
# 훈련데이터와 시험데이터 분할

from sklearn.ensemble import RandomForestRegressor

rf=RandomForestRegressor()
rf.fit(train_data, train_target)
rf.score(test_data, test_target)

0.9998188650253648

In [28]:
len(train_data)

9797

In [27]:
for i in range(len(train_date))
    print(test_target[i])
    print(rf.predict([test_data[i]]))

214
[213.46]


랜덤포레스트 회귀모델
- 교차검증

In [9]:
from sklearn.model_selection import cross_val_score

cross_score=cross_val_score(rf,data, target, cv=5)
np.mean(cross_score)

0.9992042671573289

랜덤포레스트 회귀모델
- 시뮬레이션

In [10]:
from sklearn.model_selection import GridSearchCV

dic={'n_estimators': range(10,30),'max_depth' : range(3,5)}
# RandomForestRegressor()에서 n_estimators : int, default=100 / max_depth : int, default=None
simul=GridSearchCV(estimator=rf, param_grid=dic, cv=5)
simul.fit(data, target)

best_para=simul.best_params_
simul_result=simul.cv_results_
best_estimator=simul.best_estimator_

In [11]:
best_para

{'max_depth': 4, 'n_estimators': 17}

In [12]:
df=pd.DataFrame(simul_result)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.071236,0.002181,0.002270,0.000052,3,10,"{'max_depth': 3, 'n_estimators': 10}",0.903611,0.900813,0.930115,0.942791,0.938408,0.923148,0.017594,39
1,0.080107,0.005523,0.002331,0.000046,3,11,"{'max_depth': 3, 'n_estimators': 11}",0.901112,0.897078,0.932851,0.939938,0.938139,0.921823,0.018747,40
2,0.084902,0.002902,0.002372,0.000031,3,12,"{'max_depth': 3, 'n_estimators': 12}",0.906923,0.911924,0.934162,0.943445,0.945366,0.928364,0.016001,22
3,0.091998,0.003295,0.002463,0.000039,3,13,"{'max_depth': 3, 'n_estimators': 13}",0.905379,0.901176,0.933189,0.942453,0.939601,0.924360,0.017524,36
4,0.096598,0.002813,0.002548,0.000044,3,14,"{'max_depth': 3, 'n_estimators': 14}",0.904196,0.910754,0.933004,0.943532,0.940277,0.926353,0.015921,27
5,0.106063,0.002931,0.002842,0.000103,3,15,"{'max_depth': 3, 'n_estimators': 15}",0.912769,0.906968,0.925096,0.942100,0.944189,0.926224,0.015019,29
6,0.112925,0.004935,0.002854,0.000059,3,16,"{'max_depth': 3, 'n_estimators': 16}",0.908450,0.903006,0.930833,0.943563,0.940899,0.925350,0.016664,34
7,0.119400,0.003275,0.003075,0.000103,3,17,"{'max_depth': 3, 'n_estimators': 17}",0.903646,0.898683,0.928213,0.947079,0.940872,0.923698,0.019441,38
8,0.123610,0.003225,0.003109,0.000192,3,18,"{'max_depth': 3, 'n_estimators': 18}",0.905311,0.911358,0.930158,0.943855,0.936541,0.925444,0.014752,32
9,0.130990,0.002950,0.003096,0.000060,3,19,"{'max_depth': 3, 'n_estimators': 19}",0.904769,0.913597,0.932695,0.944092,0.942988,0.927628,0.015824,23


In [13]:
best_estimator.score(test_data, test_target)

0.9832007118527049

선형 회귀 모델

In [29]:
data=[]
target=[]

for i in range(len(bike)):
    a=list(bike.iloc[i,1:11])
    b=bike.iloc[i,11]
    data.append(a)
    target.append(b)

import numpy as np
data=np.array(data)
target=np.array(target)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data,target,test_size=0.1)
# 여기까진 동일하게

In [31]:
from sklearn.linear_model import LinearRegression

lr=LinearRegression()
lr.fit(train_data, train_target)
lr.score(test_data,test_target)

1.0

In [50]:
A=lr.score(test_data,test_target)
print("{:.100f}".format(A))

1.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [44]:
lr.coef_

array([-6.22507716e-15, -6.77236045e-14, -9.71418041e-15, -7.59358939e-15,
        3.49720253e-15, -2.84494650e-15,  3.33066907e-16, -6.45317133e-16,
        1.00000000e+00,  1.00000000e+00])

In [45]:
lr.intercept_

-2.842170943040401e-14

In [54]:
from sklearn.model_selection import cross_val_score

cross_score=cross_val_score(lr,data, target, cv=10)
np.mean(cross_score)

1.0